In [2]:
import requests
import re
import pandas as pd
import folium
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In [3]:
ciudades_natales = {'ciudad': [], "pais": []}

In [6]:
art = pd.read_csv('../data/artists.csv', index_col = 0)
art.head(2)

,name,years,genre,nationality,bio,wikipedia,paintings
id,,,,,,,
0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani (Italian pronunciat...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193
1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky (Russian: Васи́...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88


In [7]:
links = art['wikipedia']

In [8]:
for url in links:
    try:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, "html.parser")
        productos = soup.find_all("div",{"class": "birthplace"})
        x = productos[0].find_all('a')

        if len(x) == 2: 
            ciudades_natales['ciudad'].append(x[0].get('title')) 
            ciudades_natales['pais'].append(x[1].get('title'))
        else:
            ciudades_natales['ciudad'].append(x[0].get('title')) 
            ciudades_natales['pais'].append('Información no disponible.')  
    except:
        ciudades_natales['ciudad'].append('Información no disponible.')
        ciudades_natales['pais'].append('Información no disponible.')


In [9]:
lugar_nacimiento = pd.DataFrame(ciudades_natales)
lugar_nacimiento.head(2)

,ciudad,pais
0,Livorno,Kingdom of Italy
1,Moscow,Russian Empire


In [10]:
ciudades = ciudades_natales['ciudad']

In [11]:
def sacar_coord(x):
    try: 
        geolocator = Nominatim(user_agent="MyApp")
        location = geolocator.geocode(x)
        return location.latitude, location.longitude
    except:
        return 'Sin información'

In [12]:
geo_ciudad = pd.DataFrame(lugar_nacimiento["ciudad"].apply(sacar_coord))

In [13]:
geo_ciudad.head(5)

,ciudad
0,"(42.790216599999994, 10.340278678735025)"
1,"(55.7504461, 37.6174943)"
2,"(19.3834636, -99.2190861)"
3,"(48.8588897, 2.3200410217200766)"
4,"(50.7123036, 3.8301131)"


In [14]:
def sacar_latitud(x):
    try:
        return float(x[0])
    except:
        return np.nan

In [15]:
geo_ciudad["latitud"] = geo_ciudad["ciudad"].apply(sacar_latitud) 

In [16]:
def sacar_longitud(x):
    try:
        return float(x[1])
    except:
        return np.nan

In [17]:
geo_ciudad["longitud"] = geo_ciudad["ciudad"].apply(sacar_longitud) 

In [18]:
geo_def = geo_ciudad.dropna()

In [19]:
mapa=folium.Map(location=[40, 15],
                tiles='openstreetmap',
                zoom_start=2)

for index, row in geo_def.iterrows():
    folium.Marker([row["latitud"], row['longitud']],
                  radius=10,
                  icon=folium.Icon(icon = "globe", icon_color = 'black', color = "lightgreen")).add_to(mapa)

mapa